In [ ]:
import numpy as np
import numba
from numba import jit

In [ ]:
@jit(nopython=True)
def ellipse( xc , yc , ax1 , ax2 , angle , X_circle , Y_circle ):

    cos_angle = np.cos(angle*np.pi/180)
    sin_angle = np.sin(angle*np.pi/180)

    # x1 = xc + ax1 * cos_angle
    # y1 = yc + ax1 * sin_angle

    # x2 = xc - ax2 * sin_angle
    # y2 = yc + ax2 * cos_angle

    X = ax1 * X_circle
    Y = ax2 * Y_circle

    xe = xc + X*cos_angle - Y*sin_angle
    ye = yc + X*sin_angle + Y*cos_angle

    return (xe,ye)

In [ ]:
xc = 288830.0
yc = 2150362.0
ax1 = 26.081118798477803
ax2 = 24.409220221209875
angle = 39.8274074028988
x_circle = np.full((30), 4.0)
y_circle = np.full((30), 3.0)
# x_circle = [ 1.0, 0.97662056,  0.90757542,  0.79609307,  0.64738628,  0.46840844,
#   0.26752834, 0.05413891, -0.161782,   -0.37013816, -0.56118707, -0.72599549,
#  -0.85685718, -0.94765317, -0.99413796, -0.99413796, -0.94765317, -0.85685718,
#  -0.72599549, -0.56118707, -0.37013816, -0.161782,    0.05413891,  0.26752834,
#   0.46840844,  0.64738628, 0.79609307,  0.90757542,  0.97662056,  1.0 ]
# y_circle = [ 0.0,  2.14970440e-01,  4.19889102e-01,  6.05174215e-01,
#   7.62162055e-01,  8.83512044e-01,  9.63549993e-01,  9.98533414e-01,
#   9.86826523e-01,  9.28976720e-01,  8.27688998e-01,  6.87699459e-01,
#   5.15553857e-01,  3.19301530e-01,  1.08119018e-01, -1.08119018e-01,
#  -3.19301530e-01, -5.15553857e-01, -6.87699459e-01, -8.27688998e-01,
#  -9.28976720e-01, -9.86826523e-01, -9.98533414e-01, -9.63549993e-01,
#  -8.83512044e-01, -7.62162055e-01, -6.05174215e-01, -4.19889102e-01,
#  -2.14970440e-01, -2.44929360e-16]

print(len(x_circle))
print(len(y_circle))

In [ ]:
ellipse(xc, yc, ax1, ax2, angle, x_circle, y_circle)

In [ ]:
%timeit ellipse(xc, yc, ax1, ax2, angle, x_circle, y_circle)

In [ ]:
%timeit ellipse.py_func(xc, yc, ax1, ax2, angle, x_circle, y_circle)

In [ ]:


# read the run parameters form the file input_data.py
@jit(nopython=True)
def load_settings(n_vents,x_vent,y_vent):
    cum_fiss_length = np.zeros(n_vents)

    for j in range(1,n_vents):

        delta_xvent =  x_vent[j] - x_vent[j-1]
        delta_yvent =  y_vent[j] - y_vent[j-1]

        cum_fiss_length[j] = cum_fiss_length[j-1] + np.sqrt( delta_xvent**2 + delta_yvent**2 )

    if ( n_vents >1 ):
        cum_fiss_length = cum_fiss_length / cum_fiss_length[j]

    return cum_fiss_length


   

In [ ]:
# filling_parameter, n_vents, cum_fiss_length = load_settings()

from input_data_advanced import *
from input_data import *


filling_parameter = 1 - thickening_parameter
n_vents = len(x_vent)

cum_fiss_length = load_settings(n_vents,np.ndarray(x_vent),np.ndarray(y_vent))

print("Timing settings loading with numba")
%timeit load_settings(n_vents,np.ndarray(x_vent),np.ndarray(y_vent))

print("Timing settings loading withOUT numba")
%timeit load_settings.py_func(n_vents,np.ndarray(x_vent),np.ndarray(y_vent))